<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Cross_Tabs_with_OLAP_Cube_PGDM_FORDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEPRECATED
see this [notebook](https://github.com/Praxis-QR/RDWH/blob/main/Cross_Tabs_%7C_OLAP_Cube_v2_PGDM_FORDS.ipynb) for updated version

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Generating Cross Tabs
Companion codes for [this slide deck](https://docs.google.com/presentation/d/1BTMkvls1VZFZf-d2MF6xj4NvpeHFNLHums5KG5Z7Ewc/edit?usp=sharing) <br>
For a more complicated version of crosstabs using Normalised Key Value Tables see this [notebook](https://github.com/Praxis-QR/RDWH/blob/c783cd84b5155577388e77b69f84bf8a17e051cd/Cross_Tabs_with_OLAP_Cube_Normalised_KeyValueTables.ipynb)

#Install MySQL

In [1]:
# Some system parameters
#!cat /proc/cpuinfo
#!cat /proc/meminfo

In [2]:
!apt-get update > /dev/null
!apt-get -y install mysql-server > /dev/null
!mysql --version
!/etc/init.d/mysql restart

mysql  Ver 8.0.34-0ubuntu0.22.04.1 for Linux on x86_64 ((Ubuntu))
 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [3]:
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


# Pandas

In [ ]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb
from sqlalchemy import create_engine, text

In [ ]:
connect_string = 'mysql://root@localhost/praxisDB'
engine = create_engine(connect_string)
#
DBConn = engine.connect()
#
df_mysql = pd.read_sql(text('show databases'), con=DBConn)
print ('loaded dataframe from MySQL. records:', len(df_mysql))
DBConn.close()
df_mysql

loaded dataframe from MySQL. records: 5


,Database
0,information_schema
1,mysql
2,performance_schema
3,praxisDB
4,sys


In [ ]:
#To run any non-SELECT SQL command
def runCMD (DDL):
    DBConn = engine.connect()
    try:
        retcode = DBConn.execute(text(DDL))
        DBConn.commit()
    except :
        print('Error : return code = ',retcode)
    DBConn.close()

#To run any SELECT SQL command and return data in pandas dataframe
def runSELECT (query):
    DBConn = engine.connect()
    df_mysql = pd.read_sql(text(query), con=DBConn)
    DBConn.close()
    return df_mysql

#Create, Load  Fact Table

In [ ]:
cmd = 'Drop table if exists CarSalesFact'
runCMD(cmd)
# ----------------------
cmd = 'Create table CarSalesFact(Make varchar(10), Type varchar(10), Colour varchar(10), Value decimal(8,0), Units int)'
runCMD(cmd)
# ----------------------

cmd = "\
INSERT INTO CarSalesFact (Make, Type, Colour, Value, Units) VALUES \
('Honda','Sedan','Brown',5000,1),('Honda','SUV','Black',18000, 4), ('Honda','SUV','Red',7000,2), \
('Mazda','Compact','Red',5000,1),('Mazda','Sedan','Brown',4000,1),('Mazda','SUV','Black',4000,1),('Mazda','SUV','Red',3000,1),\
('Toyota','Compact','Red',7000,1),('Toyota','Sedan','Black',9000,2)\
"
runCMD(cmd)
# ----------------------
#runSELECT('select * from CarSalesFact')
!mysql praxisDB -e 'select * from CarSalesFact'

+--------+---------+--------+-------+-------+
| Make   | Type    | Colour | Value | Units |
+--------+---------+--------+-------+-------+
| Honda  | Sedan   | Brown  |  5000 |     1 |
| Honda  | SUV     | Black  | 18000 |     4 |
| Honda  | SUV     | Red    |  7000 |     2 |
| Mazda  | Compact | Red    |  5000 |     1 |
| Mazda  | Sedan   | Brown  |  4000 |     1 |
| Mazda  | SUV     | Black  |  4000 |     1 |
| Mazda  | SUV     | Red    |  3000 |     1 |
| Toyota | Compact | Red    |  7000 |     1 |
| Toyota | Sedan   | Black  |  9000 |     2 |
+--------+---------+--------+-------+-------+


#Cross Tabs

In [ ]:
!mysql praxisDB -e " \
  select Make, Type, Colour, Units  \
  from CarSalesFact \
"

+--------+---------+--------+-------+
| Make   | Type    | Colour | Units |
+--------+---------+--------+-------+
| Honda  | Sedan   | Brown  |     1 |
| Honda  | SUV     | Black  |     4 |
| Honda  | SUV     | Red    |     2 |
| Mazda  | Compact | Red    |     1 |
| Mazda  | Sedan   | Brown  |     1 |
| Mazda  | SUV     | Black  |     1 |
| Mazda  | SUV     | Red    |     1 |
| Toyota | Compact | Red    |     1 |
| Toyota | Sedan   | Black  |     2 |
+--------+---------+--------+-------+


##Level 1

In [ ]:
!mysql praxisDB -e " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type \
"

+--------+---------+-------+
| Make   | Type    | Count |
+--------+---------+-------+
| Honda  | Sedan   |     1 |
| Honda  | SUV     |     6 |
| Mazda  | Compact |     1 |
| Mazda  | Sedan   |     1 |
| Mazda  | SUV     |     2 |
| Toyota | Compact |     1 |
| Toyota | Sedan   |     2 |
+--------+---------+-------+


## Level 2

In [ ]:
!mysql praxisDB -e " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type  \
"

+--------+---------+-------+
| Make   | Type    | Count |
+--------+---------+-------+
| Honda  | Sedan   |     1 |
| Honda  | SUV     |     6 |
| Mazda  | Compact |     1 |
| Mazda  | Sedan   |     1 |
| Mazda  | SUV     |     2 |
| Toyota | Compact |     1 |
| Toyota | Sedan   |     2 |
+--------+---------+-------+


##Level 2 - Rollup

In [ ]:
!mysql praxisDB -e " \
  select Make, Type, Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type  \
  with ROLLUP; \
"

+--------+---------+-------+
| Make   | Type    | Count |
+--------+---------+-------+
| Honda  | Sedan   |     1 |
| Honda  | SUV     |     6 |
| Honda  | NULL    |     7 |
| Mazda  | Compact |     1 |
| Mazda  | Sedan   |     1 |
| Mazda  | SUV     |     2 |
| Mazda  | NULL    |     4 |
| Toyota | Compact |     1 |
| Toyota | Sedan   |     2 |
| Toyota | NULL    |     3 |
| NULL   | NULL    |    14 |
+--------+---------+-------+


In [ ]:
# Cosmetic improvement on report
!mysql praxisDB -e " \
  select IFNULL(Make,'-total1-') 'Make', IFNULL(Type,'-total2-') 'Type', Sum(Units) Count \
  from CarSalesFact \
  group by Make, Type  \
  with ROLLUP; \
"

+----------+----------+-------+
| Make     | Type     | Count |
+----------+----------+-------+
| Honda    | Sedan    |     1 |
| Honda    | SUV      |     6 |
| Honda    | -total2- |     7 |
| Mazda    | Compact  |     1 |
| Mazda    | Sedan    |     1 |
| Mazda    | SUV      |     2 |
| Mazda    | -total2- |     4 |
| Toyota   | Compact  |     1 |
| Toyota   | Sedan    |     2 |
| Toyota   | -total2- |     3 |
| -total1- | -total2- |    14 |
+----------+----------+-------+


### Roll UP even more

In [ ]:
!mysql praxisDB -e " \
  select Make, Sum(Units) Count \
  from CarSalesFact \
  group by Make  \
  with ROLLUP; \
"

+--------+-------+
| Make   | Count |
+--------+-------+
| Honda  |     7 |
| Mazda  |     4 |
| Toyota |     3 |
| NULL   |    14 |
+--------+-------+


In [ ]:
!mysql praxisDB -e " \
  select IFNULL(Make,'-total-') 'Make', Sum(Units) Count \
  from CarSalesFact \
  group by Make  \
  with ROLLUP; \
"

+---------+-------+
| Make    | Count |
+---------+-------+
| Honda   |     7 |
| Mazda   |     4 |
| Toyota  |     3 |
| -total- |    14 |
+---------+-------+


In [ ]:
!mysql praxisDB -e " \
  select IFNULL(Type,'-total-') 'Type', Sum(Units) Count \
  from CarSalesFact \
  group by Type  \
  with ROLLUP; \
"

+---------+-------+
| Type    | Count |
+---------+-------+
| Compact |     2 |
| Sedan   |     4 |
| SUV     |     8 |
| -total- |    14 |
+---------+-------+


In [ ]:
!mysql praxisDB -e " \
  select Sum(Units) 'Total' \
  from CarSalesFact \
"

+-------+
| Total |
+-------+
|    14 |
+-------+


##Level 3  - THREE crosstabs

In [ ]:
!mysql praxisDB -e  "\
SELECT \
	Make,\
	SUM(IF(Type='SUV',Units,0))  'SUV',\
	SUM(IF(Type='Sedan',Units,0))  'Sedan',\
	SUM(IF(Type='Compact',Units,0))  'Compact'\
  FROM CarSalesFact\
  GROUP BY Make WITH ROLLUP\
"

+--------+------+-------+---------+
| Make   | SUV  | Sedan | Compact |
+--------+------+-------+---------+
| Honda  |    6 |     1 |       0 |
| Mazda  |    2 |     1 |       1 |
| Toyota |    0 |     2 |       1 |
| NULL   |    8 |     4 |       2 |
+--------+------+-------+---------+


In [ ]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Col_Totals') Dims,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Row_Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Type='SUV',Units,0))  'SUV',\
	SUM(IF(Type='Sedan',Units,0))  'Sedan',\
	SUM(IF(Type='Compact',Units,0))  'Compact'\
  FROM CarSalesFact\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+------------+------+-------+---------+------------+
| Dims       | SUV  | Sedan | Compact | Row_Totals |
+------------+------+-------+---------+------------+
| Honda      |    6 |     1 |       0 |          7 |
| Mazda      |    2 |     1 |       1 |          4 |
| Toyota     |    0 |     2 |       1 |          3 |
| Col_Totals |    8 |     4 |       2 |         14 |
+------------+------+-------+---------+------------+


In [ ]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Col_Totals') Dims,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Row_Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Colour='Brown',Units,0))  'Brown',\
	SUM(IF(Colour='Black',Units,0))  'Black',\
	SUM(IF(Colour='Red',Units,0))  'Red'\
  FROM CarSalesFact\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+------------+-------+-------+------+------------+
| Dims       | Brown | Black | Red  | Row_Totals |
+------------+-------+-------+------+------------+
| Honda      |     1 |     4 |    2 |          7 |
| Mazda      |     1 |     1 |    2 |          4 |
| Toyota     |     0 |     2 |    1 |          3 |
| Col_Totals |     2 |     7 |    5 |         14 |
+------------+-------+-------+------+------------+


In [ ]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Type,'Totals') Dims,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
FROM ( \
  SELECT \
	Type,\
	SUM(IF(Colour='Brown',Units,0))  'Brown',\
	SUM(IF(Colour='Black',Units,0))  'Black',\
	SUM(IF(Colour='Red',Units,0))  'Red'\
  FROM CarSalesFact\
  GROUP BY Type WITH ROLLUP\
) AS sums \
"

+---------+-------+-------+------+--------+
| Dims    | Brown | Black | Red  | Totals |
+---------+-------+-------+------+--------+
| Compact |     0 |     0 |    2 |      2 |
| Sedan   |     2 |     2 |    0 |      4 |
| SUV     |     0 |     5 |    3 |      8 |
| Totals  |     2 |     7 |    5 |     14 |
+---------+-------+-------+------+--------+


In [ ]:
# Same Query using the
# CTE structure
# ---------------------------------------------------------------------------
!mysql praxisDB -e "\
with sums as \
 ( \
  SELECT \
	Type,\
	SUM(IF(Colour='Brown',Units,0))  'Brown',\
	SUM(IF(Colour='Black',Units,0))  'Black',\
	SUM(IF(Colour='Red',Units,0))  'Red'\
  FROM CarSalesFact\
  GROUP BY Type WITH ROLLUP\
) \
SELECT \
IFNULL(Type,'Totals') Dims,\
sums.Brown, sums.Black, sums.Red,\
sums.Brown + sums.Black + sums.Red  Totals \
from sums\
"
#runSELECT(CTESQL)

+---------+-------+-------+------+--------+
| Dims    | Brown | Black | Red  | Totals |
+---------+-------+-------+------+--------+
| Compact |     0 |     0 |    2 |      2 |
| Sedan   |     2 |     2 |    0 |      4 |
| SUV     |     0 |     5 |    3 |      8 |
| Totals  |     2 |     7 |    5 |     14 |
+---------+-------+-------+------+--------+


##Level 3 A - the UNION ALL / CUBE

In [ ]:
!mysql praxisDB -e  "\
SELECT Make, Type, SUM(Units) \
FROM CarSalesFact \
GROUP BY Make, Type \
UNION ALL \
SELECT Make, 'ALL', SUM(Units) \
FROM CarSalesFact \
GROUP BY Make \
UNION ALL \
SELECT 'ALL', Type, SUM(Units) \
FROM CarSalesFact \
GROUP BY Type \
UNION ALL \
SELECT 'ALL', 'ALL', SUM(Units) \
FROM CarSalesFact\
"



+--------+---------+------------+
| Make   | Type    | SUM(Units) |
+--------+---------+------------+
| Honda  | Sedan   |          1 |
| Honda  | SUV     |          6 |
| Mazda  | Compact |          1 |
| Mazda  | Sedan   |          1 |
| Mazda  | SUV     |          2 |
| Toyota | Compact |          1 |
| Toyota | Sedan   |          2 |
| Honda  | ALL     |          7 |
| Mazda  | ALL     |          4 |
| Toyota | ALL     |          3 |
| ALL    | Sedan   |          4 |
| ALL    | SUV     |          8 |
| ALL    | Compact |          2 |
| ALL    | ALL     |         14 |
+--------+---------+------------+


#Create a 'standard' CUBE Fact table
What is [GROUP_CONCAT](https://www.google.co.in/search?client=opera&q=group_concat+in+mysql&sourceid=opera&ie=UTF-8&oe=UTF-8)?

#Cross Tab

In [ ]:
!mysql praxisDB -e "select * from CarSalesFact"

+--------+---------+--------+-------+-------+
| Make   | Type    | Colour | Value | Units |
+--------+---------+--------+-------+-------+
| Honda  | Sedan   | Brown  |  5000 |     1 |
| Honda  | SUV     | Black  | 18000 |     4 |
| Honda  | SUV     | Red    |  7000 |     2 |
| Mazda  | Compact | Red    |  5000 |     1 |
| Mazda  | Sedan   | Brown  |  4000 |     1 |
| Mazda  | SUV     | Black  |  4000 |     1 |
| Mazda  | SUV     | Red    |  3000 |     1 |
| Toyota | Compact | Red    |  7000 |     1 |
| Toyota | Sedan   | Black  |  9000 |     2 |
+--------+---------+--------+-------+-------+


##Level 1 - Partial Cross Tab

In [ ]:
!mysql praxisDB -e "\
select Make, Type, sum(Units) Count, sum(Value) Total \
from \
CarSalesFact \
group by Make, Type\
"

+--------+---------+-------+-------+
| Make   | Type    | Count | Total |
+--------+---------+-------+-------+
| Honda  | Sedan   |     1 |  5000 |
| Honda  | SUV     |     6 | 25000 |
| Mazda  | Compact |     1 |  5000 |
| Mazda  | Sedan   |     1 |  4000 |
| Mazda  | SUV     |     2 |  7000 |
| Toyota | Compact |     1 |  7000 |
| Toyota | Sedan   |     2 |  9000 |
+--------+---------+-------+-------+


##Level 2 Roll UP

In [ ]:
!mysql praxisDB -e "\
select Make, Type, sum(Units) Count, sum(Value) Total \
from \
CarSalesFact \
group by Make, Type with ROllUP\
"

+--------+---------+-------+-------+
| Make   | Type    | Count | Total |
+--------+---------+-------+-------+
| Honda  | Sedan   |     1 |  5000 |
| Honda  | SUV     |     6 | 25000 |
| Honda  | NULL    |     7 | 30000 |
| Mazda  | Compact |     1 |  5000 |
| Mazda  | Sedan   |     1 |  4000 |
| Mazda  | SUV     |     2 |  7000 |
| Mazda  | NULL    |     4 | 16000 |
| Toyota | Compact |     1 |  7000 |
| Toyota | Sedan   |     2 |  9000 |
| Toyota | NULL    |     3 | 16000 |
| NULL   | NULL    |    14 | 62000 |
+--------+---------+-------+-------+


##Level 3 Full Cross Tab

In [ ]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Totals') Dims,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Type='SUV',Units,0))  'SUV',\
	SUM(IF(Type='Sedan',Units,0))  'Sedan',\
	SUM(IF(Type='Compact',Units,0))  'Compact'\
  FROM CarSalesFact\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+--------+------+-------+---------+--------+
| Dims   | SUV  | Sedan | Compact | Totals |
+--------+------+-------+---------+--------+
| Honda  |    6 |     1 |       0 |      7 |
| Mazda  |    2 |     1 |       1 |      4 |
| Toyota |    0 |     2 |       1 |      3 |
| Totals |    8 |     4 |       2 |     14 |
+--------+------+-------+---------+--------+


In [ ]:
!mysql praxisDB -e  "\
SELECT \
IFNULL(Make,'Totals') Dims,\
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM ( \
  SELECT \
	Make,\
	SUM(IF(Type='SUV',Value,0))  'SUV',\
	SUM(IF(Type='Sedan',Value,0))  'Sedan',\
	SUM(IF(Type='Compact',Value,0))  'Compact'\
  FROM CarSalesFact\
  GROUP BY Make WITH ROLLUP\
) AS sums \
"

+--------+-------+-------+---------+--------+
| Dims   | SUV   | Sedan | Compact | Totals |
+--------+-------+-------+---------+--------+
| Honda  | 25000 |  5000 |       0 |  30000 |
| Mazda  |  7000 |  4000 |    5000 |  16000 |
| Toyota |     0 |  9000 |    7000 |  16000 |
| Totals | 32000 | 18000 |   12000 |  62000 |
+--------+-------+-------+---------+--------+


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)